In [1]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from collections import defaultdict
%matplotlib inline

In [2]:
punc = string.punctuation

In [3]:
conn = psycopg2.connect('dbname = mh')

In [4]:
cur = conn.cursor()

In [5]:
query_post = """
    Select * from mh_anx_post;
    """

query_user = """
    Select * from mh_anx_user;
    """

In [6]:
cur.execute(query_post)

In [7]:
thread = cur.fetchall()

In [8]:
cur.execute(query_user)

In [9]:
members = cur.fetchall()

In [10]:
conn.commit()

### Text Cleaning and getting into document format

### mh forum cleaning

In [11]:
# for pandas df
pid = []
user = []
post = []
mood = []
post_type = []
post_title = []
title = []
forum_name = []

#for nlp matrix
md = defaultdict(list)

#for all posts
for info in thread:
    pid.append(info[0])
    user.append(info[1])
    
    #remove extra space in document and remove punctuation and append to dictionary
    doc = info[2]
    doc = "".join(re.sub(r'\s\s', '|', doc).split())
    doc = doc.replace("|", " ")
    doc = [letter for letter in doc if letter not in punc]
    doc = ("".join(doc).lower())
    md[info[1]].append(doc)
    
    post.append(doc)
    mood.append(info[3])
    post_type.append(info[4])
    post_title.append(info[5])
    title.append(info[6])
    forum_name.append(info[7])
    
# create post df from mh forum
mh_anx_post = pd.DataFrame({'pid':pid, 'user': user, 'post': post, 'mood': mood, 'post_type': post_type, 'post_title': post_title, 'thread_title': title, 'forum_name': forum_name})
mh_anx_post = mh_anx_post[['pid', 'user', 'post_title', 'post', 'post_type', 'mood', 'thread_title', 'forum_name']]

In [13]:
#for pandas df
user = []
join_date = []

for member in members:
    user.append(member[0])
    join_date.append(member[1])
    
mh_anx_user = pd.DataFrame({'user': user, 'member_since': join_date})
mh_anx_user = mh_anx_user[['user', 'member_since']]

In [14]:
mh_anx_post.head()

,pid,user,post_title,post,post_type,mood,thread_title,forum_name
0,1592723,Andy1963,Sudden Onset Anxiety - Please help!,hi ive never suffered any sort of anxiety befo...,author,nan,Sudden Onset Anxiety - Please help!,Generalized Anxiety Disorder
1,1592736,Zardos,nan,i can sympathize i get crippling anxiety when ...,responder,nan,Sudden Onset Anxiety - Please help!,Generalized Anxiety Disorder
2,1594183,rachelangelo,nan,im sorry youre dealing with this i dont know i...,responder,nan,Sudden Onset Anxiety - Please help!,Generalized Anxiety Disorder
3,1598245,akash,nan,apart from medical help there are many tricks ...,responder,nan,Sudden Onset Anxiety - Please help!,Generalized Anxiety Disorder
4,585119,Mayfair,nan,great tune i had my beatles number one hits i...,responder,nan,didnt know where to post this,Social Anxiety


In [15]:
mh_anx_user.head()

,user,member_since
0,steviep43,Jan 2018
1,Chalmers333,Jan 2018
2,JenIAm,Jan 2018
3,Aw3092,Jan 2018
4,srussells,Jan 2017


### PC forum cleaning

In [16]:
#query from pc forum
query_post = """
    Select * from pc_anx_post;
    """

query_user = """
    Select * from pc_anx_user;
    """

In [17]:
cur.execute(query_post)
thread = cur.fetchall()

In [18]:
cur.execute(query_user)
members = cur.fetchall()

In [20]:
pid = []
user = []
mood = []
post = []
post_type = []
post_title = []
forum_name = []
title = []


for info in thread:
    pid.append(info[0])
    user.append(info[1])
    mood.append(info[2])
    
    doc = info[3]
    doc = " ".join([word.strip().lower() for word in doc.split()])
    doc = "".join([letter for letter in doc if letter not in punc])
    md['user'].append(doc) #appending to dictionary of users and docs from above
    
    post.append(doc)
    post_type.append(info[4])
    post_title.append(info[5])
    forum_name.append(info[6])
    title.append(info[7])
    
    
    
pc_anx_post = pd.DataFrame({'pid': pid, 'user': user, 'mood': mood, 'post': post, 'post_type': post_type, 'post_title': post_title, 'thread_title': title, 'forum_name': forum_name})
pc_anx_post = pc_anx_post[['pid', 'user', 'post_title', 'post', 'post_type', 'mood', 'thread_title', 'forum_name']]

In [21]:
#for pandas df
user = []
join_date = []

for member in members:
    user.append(member[0])
    join_date.append(member[1])
    
pc_anx_user = pd.DataFrame({'user': user, 'member_since': join_date})
pc_anx_user = pc_anx_user[['user', 'member_since']]

In [22]:
pc_anx_post.head()

,pid,user,post_title,post,post_type,mood,thread_title,forum_name
0,5974949,BBB2,Is SA neurological?,is there neurological basis for sa i feel like...,author,ashamed,Is SA neurological?,Social Anxiety and Selective Mutism
1,5686333,AmandaBroken,Anxiety Daily Check-In point #4,amandabroken,responder,sunshine,Anxiety Daily Check-In point #4,Anxiety Panic and Phobias
2,5686378,zzzsmokeyzzz,Anxiety Daily Check-In point #4,feeling pretty good today life is going pretty...,responder,calm,Anxiety Daily Check-In point #4,Anxiety Panic and Phobias
3,5686442,DodgersMom,Anxiety Daily Check-In point #4,anxiety is through the roof just found out my ...,responder,content,Anxiety Daily Check-In point #4,Anxiety Panic and Phobias
4,5686444,AmandaBroken,Anxiety Daily Check-In point #4,amandabroken,responder,sunshine,Anxiety Daily Check-In point #4,Anxiety Panic and Phobias


In [23]:
pc_anx_user.head()

,user,member_since
0,Nike007,Jul 2015
1,Bozdickens,Jun 2016
2,sonjaward809,Feb 2013
3,shattered sanity,Aug 2012
4,sans,May 2013


### Merging DF's

In [24]:
anx_post = pd.concat([mh_anx_post, pc_anx_post])

In [25]:
anx_user = pd.concat([mh_anx_user, pc_anx_user])

### Preparing the documents

In [73]:
user_list = [key for key in md]

In [82]:
documents = [md[user] for user in user_list]

In [84]:
documents = [" ".join(doc) for doc in documents]

# NLP

## Text Processing Pipeline

In [29]:
from nltk.corpus import stopwords

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
tfidf_vect = TfidfVectorizer(stop_words = 'english')

In [88]:
tfidf_vectorized = tfidf_vect.fit_transform(documents)

In [33]:
from sklearn.metrics.pairwise import linear_kernel

In [89]:
len(tfidf_vect.vocabulary_)

68780

In [90]:
cos_sim = linear_kernel(tfidf_vectorized, tfidf_vectorized)

In [91]:
top_match = np.argsort(cos_sim, axis = 1)

In [92]:
#remove self 
top_match = top_match[:, 0:-1]

In [104]:
def top_n_others(sim_mat, ri, n, users):
    """
    Inputs:
        - sim_mat: sorted similarity matrix with matching self removed
        - ri: list of row indices which define users to pick from  
        - n: top n most similar users
        - matches: list of users
    Outputs:
        - user, that user's top n most similar matches starting with most similar
        """
    
    top_n = list(range(-1,-n-1,-1))
    doc = sim_mat[ri, :]
    user = users[ri]
    sim_users = list(doc[top_n])
    return user, [users[sim] for sim in sim_users]

         
        
        
    

In [106]:
pint, sims = top_n_others(top_match, 0, 5, user_list)

In [115]:
print(pint)
print ("-"*((len(pint))))
print("doc:", " ".join(md[pint]))
print("-"*100)

for p in sims:
    print(p)
    print ("-"*((len(p))))
    print("doc:", " ".join(md[p]))
    print("-"*100)


Andy1963 
---------
doc: hi ive never suffered any sort of anxiety before but over the last five days i have started experiencing some weird sort of dread fear i cant pin it down to anything but it is scary its like an overwhelming feeling that envelopes me especially during the evenings like right now im sleeping fine and other than feeling a bit apprehensive getting out of bed each morning i am virtually unaffected by it during the day can anybody please give me their opinion on this
----------------------------------------------------------------------------------------------------
Acraftybird 
------------
doc: hi i am new to this forum but was wondering if anyone else feels weird kinda like they arent really here and no one around them is real i feel like im not in my body and when i go to sleep at night i feel like im gonna be gone the next day when i wake up getting back to the weird feeling though i feel all dizzy and like im gonna spaz out at any moment its a very scary feelin

First issues:
- seems like people with long documents are being highly compared to others simply because of the probability that they are going to include words that match other peoples